# DISCO: DISTRIBUTED INFERENCE WITH SPARSE COMMUNICATIONS ([ссылка](https://arxiv.org/pdf/2302.11180))

**Идея:** Сначала обучаем полную модель с плотной коммуникацией. Затем определяем ненулевые веса, соответствующие разреженным признакам для связи. Постепенно упрощаем коммуникацию, тонко настраивая оставшиеся веса. Эта среда, называемая распределенным выводом с разреженными коммуникациями (DISCO), может искать модели с лучшим компромиссом между задержкой передачи данных, задержкой вычислений и точностью. Кроме того, учитывая, что обмен данными и вычисления могут быть конвейеризированы, чтобы скрыть их задержку, предложенный метод может еще больше сократить сквозную задержку системы практически без ущерба для точности.

Экспериментируя с несколькими задачами машинного обучения, включая распознавание изображений, обнаружение объектов, семантическую сегментацию и сверхразрешение, мы пришли к выводу, что:
1) по сравнению с DNN с полностью независимыми ветвями, точность может быть значительно увеличена за счет очень небольшой доли передачи данных
2) По сравнению с DNN с полностью зависимыми слоями ввода-вывода (плотная передача данных), предложенный нами метод может значительно сократить обмен данными с незначительной потерей точности.

## Постановка задачи

Основной операцией вывода модели DNN является умножение входных признаков и весов. Входные признаки представляют собой карты признаков для сверточных нейронных сетей (СНС) и нейронные векторы/матрицы для многослойных восприятий (MLP) и трансформеров. Предположим, что есть I - входные объекты. Распределим вычисления каждого слоя DNN на N узлов, они равномерно распределяются с $\frac{I}{N}$ входными признаками на узел. Если выходной признак в Node-i зависит от входных признаков в Node-j, соответствующие зависимые входные признаки должны передаваться от Node-j к Node-i, индуцируя обмен данными от Node-j к Node-i. 

На рисунке ниже показан пример распределения сверточного слоя с 3 × 3 ядрами на N = 2 узла. Общее количество входных признаков равно I = 16, и каждый узел несет 8 из них. Для обычных DNN с плотными коммуникациями, тензор свертки на каждом узле имеет вид $(W, H, I, O) = (3, 3, 16, 8)$, где W и H — размер ядра, I и O — 16 входных признаков и 8 выходных признаков соответственно. Таким образом, общее количество выходных карт признаков составляет 8 + 8 = 16. Несмотря на то, что вычисление операции свертки распределено между 2 узлами, задержка связи может быть узким местом системы, поскольку все 8 входных функций должны передаваться по каналу связи в обоих направлениях. С другой стороны, в DISCO, чтобы уменьшить задержку связи, например, Node-0 может выбрать 3 входных признака, которые являются 2-й, 3-й и 6-й картами признаков, и отправить их на Node-1, что приводит к сокращению на $\frac{5}{8}$ времени связи от изначального. Аналогично, в обратном направлении, выбор 0-го и 5-го сопоставлений признаков с Node-1 на Node-0 может уменьшить передачу данных на $\frac{6}{8}$ от первоноального.

![NodeInteractionDISCO](./pictures/DISCO/NodeInteractionDISCO.png)

Задача проектирования модели DNN для распределенного вывода может быть сформулирована в виде следующих нескольких вопросов:

1) Если известно общее количество признаков, подлежащих передаче, то какое подмножество входных признаков следует выбрать для передачи на другие узлы?
2) Если подмножество передаваемых входных признаков определено, то как следует обучать веса DNN?
3) Сколько входных функций должно быть передано между узлами?

## Выбор подмножества входных признаков для взаимодействия

**Замечание:** Для примера взят сверточный слой, но это также обобщается на другие слои

Пусть $I_{each} = \frac{I}{N}$ — количество локальных входных признаков на каждом узле, и предположим, что количество входных признаков, переданных от Node-i к Node-j, равно $I_{comm}$, тогда $\binom{I_{comm}}{I_{each}}$ - возможные подмножества входных объектов для выбора. Это NP-трудная задача. Например, если $I_{each} = 128$ и $I_{comm} = 32$, то количество выборок подмножества больше 1030, что делает невозможным перечисление.

Пусть подмножество входных признаков, которые не передаются другому узлу, обозначается $F$, математически это эквивалентно передаче их как всех нулевых признаков $F_{zero}$, и соответствующие веса $W_F$, свертывающие эти все нулевые признаки, могут быть произвольными, рисунок (a). Это связано с тем, что ноль умножить на любое значение равно нулю. Тогда математически эквивалентно передать все входные функции, включая $F$, и установить $W_F$ = 0, как показано на рисунке (b). Таким образом, задача выбора подмножества признаков, который является динамическим и связан с распределением набора данных, может быть преобразована в задачу обрезки веса, где шаблон обрезки представляет собой последовательность ядер свертки (например, последовательность из 3 × 3 ядер), которая соответствует входным признакам в других узлах. Следовательно, если тензор веса 4-D свертки проецируется на 2-D матрицу-маску путем сжатия каждого ядра 2-D свертки в одно значение, указывающее, является ли это ядро полностью нулевым, то обрезанный шаблон представляет собой половину строки матрицы (для 2 узлов) или, в общем случае, $\frac{1}{N}$ строку матрицы, ибо каждая подстрока представляет веса, которые свертывают один входной признак в других узлах. Важно, что для узла Node-i всегда сохраняются ядра свертки, соответствующие его входным признакам. Поэтому диагональные блоки матрицы маски всегда сохраняются. На рисунке (c) показана двухузловая матрица масок для одного из слоев в ResNet-50, который имеет 256 входных и выходных карт признаков соответственно. Разреженность коммуникаций составляет 90%. Каждая точка в маске представляет собой сверточное ядро 3 × 3. Два темных блока по диагонали представляют плотные веса и коммуникации внутри каждого узла. В антидиагонали белые области означают, что соответствующие веса обрезаны, а темно-черные подстроки соответствуют тем входным признакам, которые будут передаваться между двумя узлами.

![WeightSelection](./pictures/DISCO/WeightSelection.png)

**Пояснение(4-D с 2-D маской):** $2-D$ маска - матрица, показывающая, какие веса у нас обнуляются, а какие нет. Например, у нас есть 2 входных и 4 выходных слоя. Тогда получается матрица вида: 
$$
\begin{equation*}
\left(
\begin{array}{cccc}
1 & 1 & 0 & 0 \\
0 & 0 & 1 & 1
\end{array}
\right)
\end{equation*}
$$  
1 - показывает, что соответствующая матрица не обнулена, 0 - что обнулена. 

После преобразования задачи выбора входного признака в задачу обрезки весов мы можем эффективно найти аппроксимацию задачи, оценив значимость весов. Сначала обучается полная модель DNN с плотными коммуникациями. Затем для каждого слоя создадим матрицу масок $M \in \{0, 1\}^{I*O}$ для каждого сверточного ядра размерности $W×H$, где I и O — количество входных и выходных признаков соответственно. Затем для недиагональной подстроки размером 1-на-$\frac{O}{N}$, которая соответствует $\frac{OWH}{N}$ весам в полной модели, вычисляется для них норма $L_1$ и обрезаются подстроки с нормами меньшими $L_1$(заданный порог) обрезаются. Если мы разделим маску $M$ на $\frac{I}{N} × \frac{O}{N}$ подматриц, то обрезка одной недиагональной подстроки в (i, j)-й подматрице соответствует предотвращению передачи одного входного признака из Node-i в Node-j. Общее количество обрезанных подстрок зависит от желаемой разреженности в недиагональной матрице, которая является разреженностью связи этого слоя. Остальные подстроки являются входными признаками с большей значимостью и должны передаваться между узлами для оптимального соотношения задержки и точности. Мы удаляем через случайную обрезку подрядков, чтобы показать преимущества DISCO

## Обучение DNN с разреженной связью

После установки эквивалентности между разреженными весами и разреженными коммуникациями мы используем простую, но эффективную итеративную процедуру обрезки величины и тонкой настройки для получения обученных DNN с разреженными коммуникациями. Сначала мы обучаем полноценную модель с плотными коммуникациями. Затем мы отсекаем наименее значимые подстроки в матрице весов на долю $p_1$, а затем тонко настраиваем оставшиеся веса, чтобы восстановить некоторую точность. На этом завершается одна итерация.Процесс обрезки и тонкой настройки выполняется в течение нескольких итераций для создания последовательности моделей с возрастающей разреженностью веса, которая соответствует моделям с возрастающей разреженностью связи и вычислений. Если мы установим целевую разреженность связи равной 100% в DISCO, то результирующая модель будет иметь ту же архитектуру, что и модель со всеми независимыми ветвями. Тем не менее, модель, полученная с помощью DISCO, значительно превосходит модели прямого обучения со всеми независимыми ветвями, при одинаковом количестве обучающих усилий (например, эпох).

## Количество входных features для взаимодействия

Общая задержка системы зависит от задержки вычислений и связи. Предположим, что обычный слой DNN имеет вычислительную сложность $C_c$ (число операция с плавающей запятой (FLOPs)). и общий размер features составляет $F_s$ (в байтах) Если в коммуникациях нет разреженности, каждый узел будет отправлять свои собственные входные признаки размера $\frac{F_s}{N}$ и получать все остальные функции от других узлов общего размера $\frac{F_s(N-1)}{N}$ Таким образом, общая передача данных каждым узлом равна $\frac{F_s}{N} + \frac{F_s(N-1)}{N} = F_s$.

Чтобы избежать сложностей с добавлением дополнительных параметров системы, мы предполагаем равную пропускную способность получения/отправки, обозначаемую B (в байтах/секунду). Пусть $S_{comm}$ обозначает разреженность связи каждого узла, тогда задержка коммуникаций равна:

$$
L_{comm} = \frac{F_s(1-S_{comm})}{B} \ (1)
$$

Пусть $S_{comp}$ обозначает разреженность вычислений всего слоя (которая также равна разреженности веса $S_{wt}$), а C обозначает скорость вычислений (в операциях в секунду), тогда задержка вычислений для каждого узла равна:

$$
L_{comp} = \frac{C_c(1-S_{comp})}{NC} \ (2)
$$

На рисунке ниже показаны $L_{comm}$ и $L_{comp}$ для каждого уровня модели ResNet-50 с плотной связью $(S_{comp} = S_{comm} = 0)$, когда вывод распределяется на $N = 2$ узла. Мы предполагаем, что каждое значение в картах признаков представлено в виде 4-байтового числа с плавающей запятой.

![Resnet](./pictures/DISCO/L_commAndL_comp.png)

**Замечание:** ось Y находится в логарифмической шкале, и задержка связи может быть больше задержки вычислений на один или два порядка.

**Замечание:** Из самого вервого рисунка видно, что разреженная коммуникация также может привести к сокращению вычислений. Отношения можно описать следующим образом:

$$S_{comm}=\frac{N}{N-1}S_{comp} \ (3)$$

Доказательство:

Пусть $(W, H, I, O)$ обозначает форму тензора свертки одного плотного слоя с плотными коммуникациями. Когда этот плотный слой равномерно распределен по $N$ узлам, тензор свертки в каждом узле будет иметь форму $(W, H, I, O N )$. Среди $I$ входных признаков $\frac{I}{N}$ из локальных узлов, а другой $\frac{(N−1)I}{N}$ передается из других узлов. Если мы уменьшим количество **передаваемых** feauters до $I'$, то сокращение связи составит:
$$
S_{comm} = 1 - \frac{I'}{\frac{(N-1)I}{N}} = \frac{NI-I-I'N}{(N-1)I} \ (*)
$$

С другой стороны, при передаче $I′$ признаков узел имеет $\frac{I}{N + I′}$ признаков в качестве входных данных для своего тензора свертки, и поэтому форма тензора свёртки равна $(W, H, \frac{I}{N + I′}, \frac{O}{N})$. Вычислительную редукцию этого слоя мы можем рассчитать как:

$$
S_{comp} = 1 - N\frac{(\frac{I}{N} + I')\frac{O}{N}}{IO}=\frac{NI-I-I'N}{NI} \ (**)
$$

Из $(*)$ и $(**)$ можно сделать вывод, что:
$$S_{comm}=\frac{N-1}{N}S_{comp}$$

### Конвейерная обработка коммуникаций и вычислений

Вычисления состоят из небольшой части сообщаемых features и большой части локальных features. Таким образом, асинхронные вычисления и обмен данными могут быть конвейеризированы, чтобы «скрыть» задержку друг друга. Например, один узел может одновременно начать обработку свертки признаков и собственные веса и получение признаков от другого узла. Следовательно, в режиме «конвейера» задержка вывода уровня $i$ — это максимальная задержка вычислений или связи, т. е.

$$
L(i)=\{L_{comm}(i), L_{comp}(i)\} (4)
$$

где $L_{comm}(i)$ и $L_{comp}(i)$ обозначают соответствующую задержку Layer-i и могут быть вычислены по уравнениям 1 и 2.
Режим «ожидания» (без pipeline) означает, что вычисления начинаются после завершения взаимодействия с признаками $(L(i) = L_{comm}(i) + L_{comp}(i))$. Общая задержка DNN в этом случае является суммой задержок всех уровней.

### Вычислительные и коммуникационные равновесия

Из уравнения 4 следует, что задержка слоя определяется более медленной задержкой между $L_{comm} и $L_{comp}$. Исходя из уравнений 1, 2 и 3, как $L_{comm}$, так и $L_{comp}$ будут уменьшены за счет увеличения $S_{comm}$. Следовательно, существует точка равновесия $S^{eql}_{comm}$, где $L_{comm} = L_{comp}$. Если $S_{comm} < S^{eql}_{comm}$, уменьшение задержки слоя происходит за счет уменьшения $L_{comm}$. C другой стороны, если $S_{comm} > S^{eql}_{comm}$, дальнейшее уменьшение ${L_comm} приведет к менее эффективному компромиссу между точностью и задержкой, чем раньше, потому что $L_{comp}$ уменьшается медленнее, чем $L_{comm}$ за пределами точки равновесия, и он доминирует над задержкой слоя. Установив $L_{comm} = L_{comp}$ в уравнениях 1, 2, 3, мы можем получить равновесную разреженность связи для каждого слоя как:

$$
S^{eql}_{comm}=\frac{AN-N}{AN-N+1}
$$
где $A=\frac{NC}{C_c}\frac{F_s}{B}$

**Замечание:** $S^{eql}_{comm} \in [0,1] if A \geq 1$ => узким местом системы является задержка связи